# Collecting Features of Owner's Face...

In [ ]:
import cv2
import numpy as np
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        #print(count)
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        file_name_path = './faces/charchit/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)       
    else:
        print("Face not found")
        pass
    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# Training the Model...

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
data_path = './faces/charchit/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
Training_Data, Labels = [], []
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
Labels = np.asarray(Labels, dtype=np.int32)
charchit_model  = cv2.face_LBPHFaceRecognizer.create()
charchit_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

# Face Detection and Recognition...
# When Face is Recognised send Email and Whatsapp message
# When Face is not Recognised Launch EC2 instance and attach an EBS volume to it.

In [ ]:
import cv2
import numpy as np
import os
import pywhatkit
import smtplib
import time
import boto3
import getpass
import json, subprocess

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

def SendMail():
    msg = smtplib.SMTP("smtp.gmail.com",587)
    msg.starttls()
    paswd=getpass.getpass("password:")
    msg.login("chandrakantd077@gmail.com",  paswd)
    msg.sendmail("chandrakantd077@gmail.com","chandrakantdubey2002@gmail.com","I logged in into my system.")

def SendWhatsAppMsg():
    tm = time.localtime()
    time_hour = tm.tm_hour
    time_min = tm.tm_min
    pywhatkit.sendwhatmsg("+918329196880","I logged in into my system now.",time_hour,time_min + 2)

def CreateEC2volume():
    #os.system("aws configure")
    r=boto3.client("ec2")
    r.run_instances(ImageId="ami-011c99152163a87ae",
                    MinCount=1,
                    MaxCount=1,
                    Placement={
          'AvailabilityZone': 'ap-south-1a',},
                    InstanceType="t2.micro",
                    KeyName="My_AWS_Key_Pair",)
    
    print("instance launched")
    x = int(input("index of instance to be used: "))
    instance_id = boto3.client("ec2").describe_instances()['Reservations'][x]['Instances'][0]['InstanceId']
    zone = boto3.client("ec2").describe_instances()['Reservations'][x]['Instances'][0]['Placement']['AvailabilityZone']
    #os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone "+ zone)
    ec2_client=boto3.client("ec2")
    ec2_client.create_volume(
      AvailabilityZone='ap-south-1a',
      Encrypted=True,
      Size=10,
      VolumeType='gp2',
      TagSpecifications=[
          {
              'ResourceType': 'volume',
              'Tags': [
                  {
                      'Key': 'Name',
                      'Value': 'Task-6'
                  },
              ]
          },
      ],)

    print("EBS launched")
    y = int(input("index of volume to be used: "))
    vol_id = boto3.client("ec2").describe_volumes()['Volumes'][y]['VolumeId']
    ec2_client.attach_volume(Device='/dev/xvdb',
                       InstanceId=instance_id,
                       VolumeId=vol_id
                       )
    #os.system("aws ec2 attach-volume --volume-id "+ vol_id +" --instance-id " + instance_id + " --device /dev/xvdb")
    print("attached")
    
cap = cv2.VideoCapture(0)
counting=0
while True:
    ret, frame = cap.read()    
    image, face = face_detector(frame)    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = charchit_model.predict(face)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'  
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)   
        #time.sleep(10)
        if confidence > 90:
            cv2.putText(image, "Hey Charchit", (250, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            print("Hi!!! Charchit I recognized You!!!")
            
            SendMail()
            print("Mail Sent!!!")
            
            SendWhatsAppMsg()
            print("WhatsApp Message Sent!!!")
            cap.release()
            cv2.destroyAllWindows()
            break         
        else:
            cv2.putText(image, "I dont know, who r u", (200, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition',image)
            CreateEC2volume()
            cap.release()
            cv2.destroyAllWindows()
            break
            
    except:
        cv2.putText(image, "No Face Found", (220, 40) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for Owner's face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        cap.release()
        cv2.destroyAllWindows() 
        break
cap.release()
cv2.destroyAllWindows()